In [ ]:
import asyncio
import os
import time
import random
from typing import Dict, List, Optional, Any, TypedDict
from urllib.parse import urlparse

# Web scraping
from playwright.async_api import async_playwright, Browser, BrowserContext, Page

# LangChain imports
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.document_loaders import AsyncHtmlLoader
from langchain_community.document_transformers import MarkdownifyTransformer, Html2TextTransformer
from langchain_openai import ChatOpenAI
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.\n",	  },
)

# Environment
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ 모든 패키지가 성공적으로 import되었습니다.")

In [ ]:
# OpenAI API 키 확인
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY가 .env 파일에 설정되지 않았습니다.")

# LLM 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1,
    max_tokens=4000
)

In [ ]:
# User Agents for web scraping
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
]

In [ ]:
async def get_cleaned_html(url: str) -> str:
    """
    웹 페이지에서 핵심 콘텐츠만 추출하여 HTML을 반환하는 함수
    불필요한 요소들(네비게이션, 광고, 푸터 등)을 제거
    """    
    try:
        async with async_playwright() as p:
            # 브라우저 실행 (headless 모드)
            browser = await p.chromium.launch(
                headless=True,
                args=[
                    '--no-sandbox',
                    '--disable-dev-shm-usage',
                    '--disable-gpu',
                    '--disable-web-security',
                ]
            )
            
            # 새 페이지 생성
            context = await browser.new_context(
                user_agent=random.choice(USER_AGENTS),
                viewport={'width': 1920, 'height': 1080}
            )
            page = await context.new_page()
            
            # 페이지 로딩
            await page.goto(url, wait_until='networkidle', timeout=30000)
            
            # 페이지가 완전히 로드될 때까지 잠시 대기
            await asyncio.sleep(2)
            
            # 불필요한 요소들 제거
            await remove_unwanted_elements(page)
            
            # 정리된 HTML 콘텐츠 추출
            html_content = await page.content()
            
            # 브라우저 종료
            await browser.close()
            
            print(f"✅ 정리된 HTML 콘텐츠 추출 완료 (길이: {len(html_content):,} 문자)")
            return html_content
            
    except Exception as e:
        error_msg = f"웹 페이지 로딩 실패: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg

async def remove_unwanted_elements(page: Page):
    """
    페이지에서 불필요한 요소들을 제거하는 함수
    """
    
    # 제거할 요소들의 선택자 목록
    unwanted_selectors = [
        # 네비게이션 및 헤더/푸터
        'nav', 'header', 'footer', '.header', '.footer', '.navigation', '.navbar',
        '.menu', '.site-header', '.site-footer', '.top-bar', '.bottom-bar',
        
        # 사이드바 및 보조 콘텐츠
        'aside', '.sidebar', '.side-panel', '.widget-area', '.secondary',
        
        # 광고 및 프로모션
        '.advertisement', '.ads', '.ad', '.sponsored', '.promotion', '.banner',
        '[class*="ad-"]', '[id*="ad-"]', '[class*="ads-"]', '[id*="ads-"]',
        
        # 소셜 미디어 및 공유 버튼
        '.social', '.share', '.sharing', '.social-share', '.share-buttons',
        '.social-links', '.social-icons',
        
        # 댓글 및 관련 글
        '.comments', '.comment', '.related', '.recommendations', '.suggested',
        '.similar-posts', '.more-posts',
        
        # 구독 및 뉴스레터
        '.newsletter', '.subscribe', '.subscription', '.signup',
        
        # 기타 불필요한 요소들
        '.cookie-notice', '.cookie-banner', '.popup', '.modal', '.overlay',
        '.breadcrumb', '.breadcrumbs', '.tags-list', '.post-meta-bottom',
        '.author-bio', '.author-info',
        
        # Medium 특화 불필요 요소들
        '.meteredContent', '.mobile-topbar', '.top-signup-bar', 
        '[data-testid="top-nav-bar"]', '[data-testid="post-sidebar"]',
        '.pw-post-body-paragraph + .pw-post-body-paragraph .highlight-menu',
        
        # 기타 일반적인 불필요 요소들
        'script', 'style', 'noscript', '.hidden', '[style*="display: none"]'
    ]
    
    # 각 선택자에 대해 요소 제거 실행
    for selector in unwanted_selectors:
        try:
            await page.evaluate(f"""
                const elements = document.querySelectorAll('{selector}');
                elements.forEach(el => el.remove());
            """)
        except Exception as e:
            # 일부 선택자가 실패해도 계속 진행
            pass
    
    # 빈 요소들 정리
    await page.evaluate("""
        // 빈 div, span 등 제거
        const emptyElements = document.querySelectorAll('div, span, p, section, article');
        emptyElements.forEach(el => {
            if (!el.textContent.trim() && !el.querySelector('img, video, audio, iframe')) {
                el.remove();
            }
        });
        
        // 너무 짧은 텍스트 요소 제거 (5글자 미만)
        const shortTextElements = document.querySelectorAll('p, div, span');
        shortTextElements.forEach(el => {
            if (el.textContent.trim().length < 5 && !el.querySelector('img, video, audio, iframe, a')) {
                el.remove();
            }
        });
    """)
    
    print("🧹 불필요한 요소들 제거 완료")

print("✅ get_cleaned_html 및 remove_unwanted_elements 함수 정의 완료")

In [ ]:
url = 'https://medium.com/musinsa-tech/10%EC%B4%88-%ED%83%80%EC%9E%84%EC%95%84%EC%9B%83%EC%97%90%EC%84%9C-%EB%B2%97%EC%96%B4%EB%82%98%EA%B8%B0%EA%B9%8C%EC%A7%80%EC%9D%98-%EC%97%AC%EC%A0%95-a58eb8faca36?source=rss----f107b03c406e---4'


In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,  # A synchronous browser is available, though it isn't compatible with jupyter.\n",	  },
)
# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

In [ ]:
# 개선된 함수로 테스트
cleaned_html = await get_cleaned_html(url)
print(f"원본 길이 vs 정리된 길이: {len(html):,} -> {len(cleaned_html):,}")
print("\n=== 정리된 HTML 미리보기 ===")
print(cleaned_html[:1500])

In [ ]:
def parsing_md(html_content: str) -> str:
    """
    HTML 콘텐츠를 Markdown으로 변환하는 함수
    LangChain MarkdownifyTransformer를 사용
    """
    
    if not html_content:
        error_msg = "HTML 콘텐츠가 없습니다."
        print(f"❌ {error_msg}")
        return error_msg
    
    print("📝 HTML → Markdown 변환 시작")
    
    try:
        # LangChain Document 객체 생성
        doc = Document(page_content=html_content)
        
        # MarkdownifyTransformer 초기화 및 변환
        transformer = MarkdownifyTransformer()
        transformed_docs = transformer.transform_documents([doc])
        
        # Markdown 콘텐츠 추출
        markdown_content = transformed_docs[0].page_content
        
        # 불필요한 공백 및 줄바꿈 정리
        lines = markdown_content.split('\n')
        cleaned_lines = []
        
        for line in lines:
            line = line.strip()
            if line:  # 빈 줄이 아닌 경우만 추가
                cleaned_lines.append(line)
        
        # 정리된 Markdown 콘텐츠
        cleaned_markdown = '\n\n'.join(cleaned_lines)
        
        print(f"✅ Markdown 변환 완료 (길이: {len(cleaned_markdown):,} 문자)")
        return cleaned_markdown
        
    except Exception as e:
        error_msg = f"Markdown 변환 실패: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg

print("✅ parsing_md 함수 정의 완료")

In [ ]:
md_content = parsing_md(html)
print(md_content[:1000])

In [ ]:
def summarize_content(md_content: str) -> str:
    """
    Markdown 콘텐츠의 핵심 내용을 요약하는 함수
    """
    
    if not md_content:
        error_msg = "Markdown 콘텐츠가 없습니다."
        print(f"❌ {error_msg}")
        return error_msg
    
    print("📄 콘텐츠 요약 시작")
    
    try:
        # 요약 프롬프트 템플릿
        summary_prompt = ChatPromptTemplate.from_template(
            """다음 웹 페이지 콘텐츠를 분석하여 핵심 내용을 요약해주세요.
            
웹 페이지 콘텐츠:
{content}

요약 지침:
1. 주요 내용과 핵심 포인트를 3-5개의 불릿 포인트로 요약
2. 기술적 내용이 있다면 중요한 기술 스택이나 개념 포함
3. 실용적이고 유용한 정보 위주로 요약
4. 200-500자 내외로 간결하게 작성
5. 한국어로 작성

요약:"""
        )
        
        # 요약 체인 생성
        summary_chain = summary_prompt | llm | StrOutputParser()
        
        # 콘텐츠 길이 제한 (토큰 수 고려)
        max_length = 8000
        if len(md_content) > max_length:
            truncated_content = md_content[:max_length] + "... (내용 일부 생략)"
        else:
            truncated_content = md_content
        
        # 요약 실행
        summary = summary_chain.invoke({"content": truncated_content})
        
        print(f"✅ 요약 완료 (길이: {len(summary)} 문자)")
        
        return summary
        
    except Exception as e:
        error_msg = f"요약 생성 실패: {str(e)}"
        print(f"❌ {error_msg}")
        
        return error_msg

print("✅ summarize_content 함수 정의 완료")

In [ ]:
summary = summarize_content(md_content)
print(summary)

In [ ]:
# PlaywrightBrowserToolkit으로 HTML 추출 테스트
toolkit_html = get_html_with_toolkit(url)
print(f"\\nToolkit으로 추출한 HTML 길이: {len(toolkit_html):,} 문자")
print(f"\\n=== Toolkit HTML 미리보기 ===")
print(toolkit_html[:1000])

In [ ]:
# PlaywrightBrowserToolkit을 사용한 새로운 HTML 추출 함수
async def get_html_with_toolkit(url: str) -> str:
    """
    PlaywrightBrowserToolkit을 사용하여 HTML 콘텐츠를 추출하는 함수
    """
    try:
        # navigate_browser 도구 찾기
        navigate_tool = None
        get_elements_tool = None
        
        for tool in tools:
            if 'navigate' in tool.name.lower():
                navigate_tool = tool
            elif 'get_elements' in tool.name.lower():
                get_elements_tool = tool
        
        if navigate_tool:
            # 페이지로 이동
            result = await navigate_tool.arun({"url": url})
            print(f"✅ 페이지 이동 결과: {result}")
            
            # HTML 콘텐츠 추출 (body 전체)
            if get_elements_tool:
                html_result = await get_elements_tool.arun({"selector": "body", "attributes": ["outerHTML"]})
                print(f"✅ HTML 추출 완료")
                return str(html_result)
            else:
                print("❌ get_elements 도구를 찾을 수 없습니다")
                return ""
        else:
            print("❌ navigate 도구를 찾을 수 없습니다")
            return ""
            
    except Exception as e:
        print(f"❌ PlaywrightBrowserToolkit 사용 실패: {e}")
        return ""

print("✅ PlaywrightBrowserToolkit 기반 함수 정의 완료")

In [ ]:
result = await get_html_with_toolkit(url)
result

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter


# 뉴스기사의 본문을 Chunk 단위로 쪼갬
text_splitter = CharacterTextSplitter(        
    separator="\n\n",
    chunk_size=3000,     # 쪼개는 글자수
    chunk_overlap=300,   # 오버랩 글자수
    length_function=len,
    is_separator_regex=False,
)

loader = WebBaseLoader(url)
loader.requests_kwargs = {'verify':False}


docs = loader.load_and_split(text_splitter)
docs

In [ ]:
print(len(docs[0].page_content))
print(docs[0].page_content)



In [ ]:
docs[0].metadata


In [62]:

urls=[
  "http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/",  
  "https://medium.com/musinsa-tech/무진장을-맞아-후기-응답속도를-개선해보자-c92e0ae60f1e",
  "https://medium.com/daangn/ai-툴-개발은-처음이라-당근-비개발자-구성원들의-ai-도전기-fb62d2a6c2f3",
  "https://tech.socarcorp.kr/fe/2025/06/10/monorepo-ci-cd-pipeline.html",
  "https://d2.naver.com/news/6518915",
  "https://meetup.nhncloud.com/posts/394",
  "https://tech.kakao.com/posts/724",
  "https://techblog.lycorp.co.jp/ko/extracting-trending-keywords-from-openchat-messages",
  "https://developers.kakaomobility.com/docs/techblogs/address-structure-1/",
  "https://oliveyoung.tech/2025-09-08/gms-qa-strategy/",
  "https://toss.tech/article/tosspeople_hyunjung",
  "https://techblog.gccompany.co.kr/%EC%97%AC%EA%B8%B0%EC%96%B4%EB%95%8C-ci-cd-%EA%B0%9C%EC%84%A0%EA%B8%B0-part-5-slack%EC%9C%BC%EB%A1%9C-%EC%99%84%EC%84%B1%EB%90%98%EB%8A%94-%EB%B0%B0%ED%8F%AC-%EA%B0%80%EC%8B%9C%EC%84%B1-c38215b4ed61?source=rss----18356045d353---4",
  "https://techblog.woowahan.com/22767/",
]

In [ ]:
loader = WebBaseLoader(urls)
loader.requests_kwargs = {'verify':False}

docs = loader.load_and_split(text_splitter)


/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'helloworld.kurly.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'medium.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/apple/IdeaProjects/techbloghub/python/venv/lib/python3.11/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'medium.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advan

In [61]:
print("=== WebBaseLoader로 여러 사이트 동시 테스트 결과 ===\\n")

for i, doc in enumerate(docs):
    url = doc.metadata.get('source', 'Unknown URL')
    content_length = len(doc.page_content)
    
    print(f"🌐 사이트 {i+1}: {url}")
    print(f"   📏 콘텐츠 길이: {content_length:,} 문자")
    print(f"   📄 메타데이터: {doc.metadata}")
    
    # 콘텐츠 미리보기 (처음 200자)
    preview = doc.page_content[:200].replace('\\n', ' ').strip()
    print(f"   👀 미리보기: {preview}...")
    
    print("-" * 80)

print(f"\\n📊 총 {len(docs)} 개 사이트에서 콘텐츠 추출 완료")

=== WebBaseLoader로 여러 사이트 동시 테스트 결과 ===\n
🌐 사이트 1: http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/
   📏 콘텐츠 길이: 2,991 문자
   📄 메타데이터: {'source': 'http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/', 'title': '우리 팀에도 Jarvis 가 생겼다 – 생성형 AI 로 만든 에러 분석가 이야기 - 컬리 기술 블로그', 'language': 'ko'}
   👀 미리보기: 우리 팀에도 Jarvis 가 생겼다 – 생성형 AI 로 만든 에러 분석가 이야기 - 컬리 기술 블로그


Kurly Tech Blog

우리 팀에도 Jarvis 가 생겼다 – 생성형 AI 로 만든 에러 분석가 이야기 
팀 내에서 생성형 AI 를 활용한 사례를 공유합니다.

김성준
게시 날짜: 2025.07.01.

왜 에러 로그는 항상 늦게 처리되는가?
에...
--------------------------------------------------------------------------------
🌐 사이트 2: http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/
   📏 콘텐츠 길이: 2,590 문자
   📄 메타데이터: {'source': 'http://thefarmersfront.github.io/blog/2025-delivery-jarvis-story/', 'title': '우리 팀에도 Jarvis 가 생겼다 – 생성형 AI 로 만든 에러 분석가 이야기 - 컬리 기술 블로그', 'language': 'ko'}
   👀 미리보기: MCP란 무엇인가? 우리는 왜 Jarvis 를 투명인간 취급하였을까

Jarvis 를 실무에 적용한 지 시간이 지나면서, 우리는 어느 순간부터 Jarvis 의 제안을 잘 참고하지

In [55]:

from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
# 각 Chunk 단위의 템플릿
template = '''다음의 내용을 한글로 요약해줘:

{text}
'''

# 전체 문서(혹은 전체 Chunk)에 대한 지시(instruct) 정의
combine_template = '''{text}

요약의 결과는 다음의 형식으로 작성해줘:
제목: 글의 제목
주요내용: 한 줄로 요약된 내용
작성자: 김철수 대리
내용: 주요내용을 불렛포인트 형식으로 작성
'''

# 템플릿 생성
prompt = PromptTemplate(template=template, input_variables=['text'])
combine_prompt = PromptTemplate(template=combine_template, input_variables=['text'])

# LLM 객체 생성
llm = ChatOpenAI(temperature=0, 
                 model_name='gpt-3.5-turbo-16k')

# 요약을 도와주는 load_summarize_chain
chain = load_summarize_chain(llm, 
                             map_prompt=prompt, 
                             combine_prompt=combine_prompt, 
                             chain_type="map_reduce", 
                             verbose=False)

chain.run(docs[0])


AttributeError: 'tuple' object has no attribute 'page_content'